In [62]:
import pandas as pd 
from gurobipy import Model, GRB, QuadExpr
import gurobipy as gb

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/price_response.csv')
df

,Product,Intercept,Sensitivity,Capacity
0,Line 1 Product 1,35234.545786,-45.896450,80020.0
1,Line 1 Product 2,37790.240832,-8.227794,89666.0
2,Line 1 Product 3,35675.333217,-7.584436,80638.0
3,Line 2 Product 1,37041.380378,-9.033166,86740.0
4,Line 2 Product 2,36846.140386,-4.427869,84050.0
5,Line 2 Product 3,35827.023747,-2.629060,86565.0
6,Line 3 Product 1,39414.266325,-2.421484,87051.0
7,Line 3 Product 2,35991.951460,-4.000512,85156.0
8,Line 3 Product 3,39313.317031,-2.296622,87588.0


Q1

In [64]:
# Define the coefficients using the DataFrame
a1 = df['Intercept'][0]  
b1 = df['Sensitivity'][0]
a2 = df['Intercept'][1]
b2 = df['Sensitivity'][1]

c1 = df['Capacity'][0]
c2 = df['Capacity'][1]

# Create a new model in Gurobi
m = Model("techessentials")

# Create variables for prices
p1 = m.addVar(lb =0, vtype=GRB.CONTINUOUS,  name ='basic price')
p2 = m.addVar(lb =0, vtype=GRB.CONTINUOUS , name= 'advance price')


d1 = m.addVar(lb=0, ub =c1, vtype=GRB.CONTINUOUS, name ='basic demand')
d2 = m.addVar(lb=0, ub =c2, vtype=GRB.CONTINUOUS, name = 'advance demand')


# Set the objective function to be maximized (revenue = price * demand)
m.setObjective(p1*(a1-b1*p1) + p2*(a2-b2*p2), GRB.MAXIMIZE)


# Add capacity constraints for demand
m.addConstr(a1 - b1 * p1 <= c1, "Capacity_Constraint_Product_1")
m.addConstr(a2 - b2 * p2 <= c2, "Capacity_Constraint_Product_2")

# Add constraint: p2 >= p1 (Advanced price should be higher than Basic)
m.addConstr(p2 >= p1, "Price_Advanced_Greater_Than_Basic")
m.addConstr(d1 >= 0, "Non_Negative_Demand_1")
m.addConstr(d2 >= 0, "Non_Negative_Demand_2")


# Optimize model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print(f'Optimal price for Product 1 (Basic): ${p1.X:.2f}')
    print(f'Optimal price for Product 2 (Advanced): ${p2.X:.2f}')
    print(f'Optimal demand for Product 1 (Basic): {d1.X:.2f} units')
    print(f'Optimal demand for Product 2 (Advanced): {d2.X:.2f} units')
    print('Maximum revenue:', m.objVal)
else:
    print('An optimal solution was not found.')

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5 rows, 4 columns and 6 nonzeros
Model fingerprint: 0x48e920bc
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [8e+04, 9e+04]
  RHS range        [4e+04, 5e+04]
Presolve removed 4 rows and 2 columns

Continuous model is non-convex -- solving as a MIP



Found heuristic solution: objective -0.0000000
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 1 bilinear constraint(s)
Found heuristic solution: objective 5.653389e+08
Variable types: 2 continuous, 0 integer (0 binary)
Found heuristic solution: objective 6.434219e+08

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 20 (of 20 available processors)

Solution count 3: 6.43422e+08 5.65339e+08 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.434218558045e+08, best bound 6.434218558045e+08, gap 0.0000%
Optimal price for Product 1 (Basic): $975.79
Optimal price for Product 2 (Advanced): $6304.94
Optimal demand for Product 1 (Basic): 80020.00 units
Optimal demand for Product 2 (Advanced): 89666.00 units
Maximum revenue: 643421855.8045


Q2